In [1]:
# automatic reloading of modules when they change.
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import draw, Dict
from qiskit_metal.qlibrary.core import BaseQubit
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from qiskit_metal.draw import LineString
from qiskit_metal.qlibrary.core.qroute import QRouteLead, QRoutePoint, QRoute
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from  qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.user_components.my_qcomponent import TransmonCrossRound,TransmonCrossRound_v1,TransmonCrossRound_v2
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from collections import  OrderedDict
import  math
import pyEPR as epr
from qiskit_metal.analyses.quantization import EPRanalysis
import numpy as np
import qiskit_metal as metal


In [3]:
# metal.about()

In [4]:
import qiskit_metal as metal
design = metal.designs.DesignPlanar()
gui = metal.MetalGUI(design)
design.chips.main.size['size_x'] = '0.754655mm'
design.chips.main.size['size_y'] = '2.0279760mm'
# design.chips.main.size['center_x'] = '-0.0060855mm'
# design.chips.main.size['center_y'] = '0.771655mm'

design.chips['main']['material'] = 'sapphire'
design.variables.cpw_gap='4 um'
design.variables.cpw_width = '8 um'
design.overwrite_enabled = True

In [5]:
# design.chips.main

In [6]:

xmon_round = TransmonCrossRound_v1(design,'my_xmon_round', options=Dict(radius = '0 um', connection_pads=dict(
                      bus_02=dict(claw_length='61um',ground_spacing='3um', claw_width='8um',claw_gap='4um',connector_location='90'),
                      )))
# xmon_round = TransmonCrossRound_v2(design,'my_xmon_round', options=Dict(cross_length='154um',radius = '0 um', connection_pads=dict(
#                       bus_02=dict(claw_length='66um',ground_spacing='3um', claw_width='8um',claw_gap='4um',connector_location='90'),
#                       )))
# Add hanger consisting of capacitively coupled transmission lines.
TQ1 = CoupledLineTee(design, 'TQ1', options=Dict(pos_x='15.02um', pos_y='1.611 mm', coupling_length='284um', coupling_space='3um',
                                                 prime_width = '10um', prime_gap = '5um', second_width = '8um', second_gap = '4um', down_length='200um',fillet='34um',mirror=True,
                                                 open_termination = False,hfss_wire_bonds=False))

pos_y = TQ1.options.pos_y

otg1 = OpenToGround(design, 'otg1', options=Dict(pos_x='-0.347mm', pos_y=pos_y,  orientation='180',width='10um',gap='5um'))
otg2 = OpenToGround(design,
                    'otg2',
                    options=Dict(pos_x='0.347mm', pos_y=pos_y,width='10um',gap='5um',))


gui.rebuild()
gui.autoscale()

In [7]:
# TQ1.pins
# xmon_round.pins

In [8]:
ops_oL = Dict(hfss_wire_bonds=False,
              pin_inputs=Dict(start_pin=Dict(component='TQ1',
                                             pin='prime_start'),
                              end_pin=Dict(component='otg1', pin='open')),trace_width='10um',trace_gap='5um')

ops_oR = Dict(hfss_wire_bonds=False,
              pin_inputs=Dict(start_pin=Dict(component='TQ1', pin='prime_end'),
                              end_pin=Dict(component='otg2', pin='open')),trace_width='10um',trace_gap='5um')

cpw_openLeft = RouteStraight(design, 'cpw_openLeft', options=ops_oL)
cpw_openRight = RouteStraight(design, 'cpw_openRight', options=ops_oR)

# -----------------------------------------------------------------------------------------------------
fillet = 0.034
# ops = Dict(fillet=fillet)

jogsS = OrderedDict()
jogsS[0] = ["R", '0um']

# spacing = 0.068000001
# spacing = 0.068001
spacing = 0.06801
qubit_cpw_length =4*design.components['my_xmon_round'].parse_options().connection_pads.bus_02.claw_width-0.004
TQ_length =design.components['TQ1'].parse_options().down_length+design.components['TQ1'].parse_options().coupling_length-2*fillet+(math.pi/2)*fillet
whole_length = 4.59754517821977
total_length = whole_length-(qubit_cpw_length+TQ_length)

start_straight = 0.15-qubit_cpw_length+fillet

l1 = start_straight-fillet
l2 = TQ1.pins.second_end.middle[1]-xmon_round.pins.bus_02.middle[1]-start_straight-13*spacing-fillet
arc_length = 14*math.pi*fillet
l3 = TQ1.parse_options().coupling_length/2-TQ1.parse_options().pos_x
excess_length = total_length-arc_length-l1-l2
asymmetry = (27*l3-28*fillet-excess_length)/28
L1 = 2*(l3-fillet-asymmetry)
L2 = l3-2*fillet-2*asymmetry

options = Dict(total_length=total_length,
                hfss_wire_bonds=False,
                pin_inputs=Dict(end_pin=Dict(component='TQ1',
                                               pin='second_end'),
                                start_pin=Dict(component='my_xmon_round', pin='bus_02')),
               meander=Dict(spacing=spacing,asymmetry=asymmetry),
                lead=Dict(start_straight=start_straight,start_jogged_extension=jogsS),
              fillet=fillet)



meanderQ= RouteMeander(design, 'meanderQ1', options=options)
gui.rebuild()
gui.autoscale()



In [ ]:
# search for freq by gradient

%%time

from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency,line_width,line_gap,N):
    [lambdaG,etfSqrt,q]=guided_wavelength(frequency*10**9,line_width*10**-6,line_gap*10**-6,500*10**-6,100*10**-9,10)
    return  lambdaG/N*10**3
# find_resonator_length(frequency=6.36,line_width=8,line_gap=4,N=4)



tar_freq= 6.66
whole_length = find_resonator_length(frequency=tar_freq,line_width=8,line_gap=4,N=4)
total_length = whole_length-(qubit_cpw_length+TQ_length)
# total_length = 4.108880921234283
# total_length1 = total_length
eps = 1.0
sim_freq = tar_freq+1e-2
n=0
# sweep_list = []



# eig_qres = EPRanalysis(design, "hfss")
# hfss = eig_qres.sim.renderer
# hfss.start()


while abs(sim_freq-tar_freq)>1e-3:
        jogsS = OrderedDict()
        jogsS[0] = ["R", '0um']

        # spacing = 0.068000001
        # spacing = 0.068001
        # spacing = 0.06801
        # qubit_cpw_length =4*design.components['my_xmon_round'].parse_options().connection_pads.bus_02.claw_width-0.004
        # TQ_length =design.components['TQ1'].parse_options().down_length+design.components['TQ1'].parse_options().coupling_length-2*fillet+(math.pi/2)*fillet
        # whole_length = 14.59754517821977
        # total_length = whole_length-(qubit_cpw_length+TQ_length)

        start_straight = 0.15-qubit_cpw_length+fillet

        l1 = start_straight-fillet
        l2 = TQ1.pins.second_end.middle[1]-xmon_round.pins.bus_02.middle[1]-start_straight-13*spacing-fillet
        arc_length = 14*math.pi*fillet
        l3 = TQ1.parse_options().coupling_length/2-TQ1.parse_options().pos_x
        excess_length = total_length-arc_length-l1-l2
        asymmetry = (27*l3-28*fillet-excess_length)/28
        L1 = 2*(l3-fillet-asymmetry)
        L2 = l3-2*fillet-2*asymmetry

        options = Dict(total_length=total_length,
                        hfss_wire_bonds=False,
                        pin_inputs=Dict(end_pin=Dict(component='TQ1',
                                                       pin='second_end'),
                                        start_pin=Dict(component='my_xmon_round', pin='bus_02')),
                       meander=Dict(spacing=spacing,asymmetry=asymmetry),
                        lead=Dict(start_straight=start_straight,start_jogged_extension=jogsS),
                      fillet=fillet)

        meanderQ= RouteMeander(design, 'meanderQ1', options=options)
        gui.rebuild()
        gui.autoscale()


        eig_qres = EPRanalysis(design, "hfss")
        hfss = eig_qres.sim.renderer
        hfss.start()
        hfss.new_ansys_design("XmonReadout_"+str(n), 'eigenmode')
        # hfss.activate_ansys_design("XmonReadout_"+str(n), 'eigenmode')  # use new_ansys_design() to force creation of a blank design
        # hfss.start()
            ### for render_design()
        # Render every QComponent in QDesign.
        render_qcomps = []

        # Identify which kind of pins in Ansys.
        # Follow details from renderer in
        # QHFSSRenderer.render_design.
        # No pins are open, so don't need to utilize render_endcaps.
        open_terminations = []

        #List of tuples of jj's that shouldn't be rendered.
        #Follow details from renderer in QHFSSRenderer.render_design.
        # render_ignored_jjs = []
        render_ignored_jjs =[('my_xmon_round', 'rect_jj')]

        # Either calculate a bounding box based on the location of
        # rendered geometries or use chip size from design class.
        box_plus_buffer = True

        # eig_qres.sim.setup.name="sweeper_em_setup"
        # eig_qres.sim.setup.min_freq_ghz=2
        # eig_qres.sim.setup.n_modes=2
        # eig_qres.sim.setup.max_passes=19
        # eig_qres.sim.setup.min_converged = 2
        # eig_qres.sim.setup.max_delta_f = 0.01

        eig_qres.setup.junctions.jj.rect = 'JJ_rect_Lj_my_xmon_round_rect_jj'
        eig_qres.setup.junctions.jj.line = 'JJ_Lj_my_xmon_round_rect_jj_'


        hfss.render_design(selection=[],
                       open_pins=[],
                       # port_list=[('cpw_openRight', 'end', 50), ('cpw_openLeft', 'end', 50)],
                       jj_to_port=[],
                       ignored_jjs=render_ignored_jjs,
                       box_plus_buffer = True)

        setup = hfss.pinfo.setup
        setup.n_modes = 2
        setup.passes = 4
        setup.delta_f=0.01
        setup.min_freq = '2GHz'

        setup.analyze()
        eig_qres.sim.convergence_t, eig_qres.sim.convergence_f, _ = hfss.get_convergences()
        sim_freq = eig_qres.sim.convergence_f['re(Mode(2)) [g]'].get(list(eig_qres.sim.convergence_f['re(Mode(2)) [g]'].keys())[-1])
        print(f"{n}th cycle is  complete, the lowest frequency is {sim_freq},the total length is {total_length}")
        tar_length = find_resonator_length(frequency=sim_freq,line_width=8,line_gap=4,N=4)
        tar_length1 = find_resonator_length(frequency=sim_freq-1e-3,line_width=8,line_gap=4,N=4)
        if abs(tar_freq-sim_freq)<0.01:
            l_r =0.5
        else:
            l_r = 1
        grad =l_r* (tar_length-tar_length1)/1e-3
        delta_length = grad*(tar_freq-sim_freq)
        total_length = total_length+delta_length
        n=n+1


# eig_qres.sim.close()





In [13]:
# eig_qres.sim.close()

In [ ]:
# sweep search freq by gradient

%%time

from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength
def find_resonator_length(frequency,line_width,line_gap,N):
    [lambdaG,etfSqrt,q]=guided_wavelength(frequency*10**9,line_width*10**-6,line_gap*10**-6,500*10**-6,100*10**-9,10)
    return  lambdaG/N*10**3
# find_resonator_length(frequency=6.36,line_width=8,line_gap=4,N=4)



tar_freq= 6.66
whole_length = find_resonator_length(frequency=tar_freq,line_width=8,line_gap=4,N=4)
total_length = whole_length-(qubit_cpw_length+TQ_length)
# total_length = 4.108880921234283
total_length1 = total_length
eps = 1.0
sim_freq = tar_freq+1e-2
n=0
sweep_list = []

while abs(sim_freq-tar_freq)>1e-3:
    eig_qres = EPRanalysis(design, "hfss")
        ### for render_design()
    # Render every QComponent in QDesign.
    render_qcomps = []

    # Identify which kind of pins in Ansys.
    # Follow details from renderer in
    # QHFSSRenderer.render_design.
    # No pins are open, so don't need to utilize render_endcaps.
    open_terminations = []

    #List of tuples of jj's that shouldn't be rendered.
    #Follow details from renderer in QHFSSRenderer.render_design.
    # render_ignored_jjs = []
    render_ignored_jjs =[('my_xmon_round', 'rect_jj')]

    # Either calculate a bounding box based on the location of
    # rendered geometries or use chip size from design class.
    box_plus_buffer = True

    eig_qres.sim.setup.name="sweeper_em_setup"
    eig_qres.sim.setup.min_freq_ghz=2
    eig_qres.sim.setup.n_modes=2
    eig_qres.sim.setup.max_passes=19
    eig_qres.sim.setup.min_converged = 2
    eig_qres.sim.setup.max_delta_f = 0.01

    eig_qres.setup.junctions.jj.rect = 'JJ_rect_Lj_my_xmon_round_rect_jj'
    eig_qres.setup.junctions.jj.line = 'JJ_Lj_my_xmon_round_rect_jj_'

    total_length1_list = []
    total_length1_list.append( total_length1)
    all_sweeps, return_code = eig_qres.run_sweep(meanderQ.name,
                                            'total_length',
                                            total_length1_list,
                                            render_qcomps,
                                            open_terminations,
                                            ignored_jjs=render_ignored_jjs,
                                            # port_list=[('cpw_openRight', 'end', 50), ('cpw_openLeft', 'end', 50)],
                                            design_name="GetEigenModeSolution",
                                            box_plus_buffer=box_plus_buffer
                                            )
    sweep_list.append(all_sweeps)
    sim_freq = all_sweeps[total_length1_list[0]]['sim_variables']['convergence_f']['re(Mode(1)) [g]'][eig_qres.sim.setup.max_passes]
    print(f"{n}th cycle is  complete, the lowest frequency is {sim_freq},the total length is {total_length1}")
    tar_length = find_resonator_length(frequency=sim_freq,line_width=8,line_gap=4,N=4)
    tar_length1 = find_resonator_length(frequency=sim_freq-1e-3,line_width=8,line_gap=4,N=4)
    if abs(tar_freq-sim_freq)<0.01:
        l_r =0.5
    else:
        l_r = 1
    grad =l_r* (tar_length-tar_length1)/1e-3
    delta_length = grad*(tar_freq-sim_freq)
    total_length1 = total_length1+delta_length
    n=n+1
    eig_qres.sim.close()



print(f"the closest frequency is {sim_freq}")

In [ ]:
# eig_qres = EPRanalysis(design, "hfss")
#
# eig_qres.sim.setup
# sweep_list[0][4.344709301045363]['sim_variables']['convergence_f']
sim_freq = all_sweeps[total_length1_list[0]]
sim_freq

In [ ]:
eig_qres.sim.close()

In [17]:
#grid search target freq

%%time
tar_freq= 6.66
total_length1_list = []
total_length2_list = []
sweep_list = []
total_length1_list.append( str(total_length)+'mm')
all_sweeps, return_code = eig_qres.run_sweep(meanderQ.name,
                                        'total_length',
                                        total_length1_list,
                                        render_qcomps,
                                        open_terminations,
                                         ignored_jjs=render_ignored_jjs,
                                        design_name="GetEigenModeSolution",
                                       box_plus_buffer=box_plus_buffer
                                      )
sweep_list.append(all_sweeps)
sim_freq = all_sweeps[total_length1_list[0]]['sim_variables']['convergence_f']['re(Mode(2)) [g]'][15]
N=0
start_length = total_length
while abs(sim_freq-tar_freq)>1e-3 and N<3:
    if sim_freq<tar_freq:
        end_length = start_length-math.pow(10,-(2+N))
        total_length2_list = np.linspace(start_length,end_length,5)
    else:
        end_length = start_length+math.pow(10,-(2+N))
        total_length2_list = np.linspace(start_length,end_length,5)
        all_sweeps, return_code = eig_qres.run_sweep(meanderQ.name,
                                        'total_length',
                                        total_length2_list,
                                        render_qcomps,
                                        open_terminations,
                                         ignored_jjs=render_ignored_jjs,
                                        design_name="GetEigenModeSolution",
                                       box_plus_buffer=box_plus_buffer
                                      )
        sweep_list.append(all_sweeps)
        sim_freq_list = []
        for i in range(5):
            sim_freq_list.append( all_sweeps[total_length2_list[i]]['sim_variables']['convergence_f']['re(Mode(2)) [g]'][15])
        sim_freq_array = np.array(sim_freq_list)
        abs_diff_array = np.absolute(sim_freq_array-tar_freq)
        sim_freq = sim_freq_array[abs_diff_array.argmin()]
        total_length2_list = []
        N= N+1


KeyboardInterrupt: 

In [9]:
# eig_qres = EPRanalysis(design, "hfss")

In [ ]:

# hfss = eig_qres.sim.renderer
# hfss.start()


In [123]:
# sim_design = hfss.pinfo.project.get_designs()[0]
# oModule = sim_design._design.GetModule("BoundarySetup")
#
# oModule.AssignImpedance(
# 	[
# 		"NAME:Imped1",
# 		"Objects:="		, ["bus_02_connector_arm_my_xmon_round","cross_my_xmon_round","ground_main_plane","prime_cpw_TQ1","second_cpw_TQ1","trace_cpw_openLeft","trace_cpw_openRight","trace_meanderQ1"],
# 		"Resistance:="		, "0",
# 		"Reactance:="		, "0.005",
# 		"InfGroundPlane:="	, True
# 	])

In [6]:
a_gds = design.renderers.gds
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [7]:
a_gds.options['short_segments_to_not_fillet'] = 'True'
a_gds.options['fabricate']='True'
scale_fillet = 2.0
a_gds.options['check_short_segments_by_scaling_fillet'] = scale_fillet
a_gds.options.negative_mask = {'main': [1,]}
# a_gds.options.cheese.shape = '1'
# a_gds.options.cheese.delta_x = '200um'
# a_gds.options.cheese.delta_y = '200um'
# a_gds.options.cheese.edge_nocheese = '200 um'
# a_gds.options.no_cheese.buffer = '50 um'
# a_gds.options.cheese.cheese_1_radius = '25 um'
a_gds.options.cheese.view_in_file =  {'main': {1: False, },}
a_gds.options.no_cheese.view_in_file =  {'main': {1: False}, }
a_gds.export_to_gds('100qubit_simulation.gds')

10:41AM 46s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"C:\Users\CJZ\PycharmProjects\resources".


1